In [1]:
import sys
sys.path.insert(0, '/das/work/p17/p17983/')

In [2]:
%matplotlib nbagg
import jungfrau_utils as ju
import numpy as np
import json
import os
import time
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

import h5py

from alvra_tools.load_data import *
from alvra_tools.channels import *

[INFO][corrections] Numba not available, reverting to Numpy


In [3]:
pede_file = "/sf/alvra/data/p17983/res/JF_pedestals/pedestal_20190728_1644.JF02T09V02.res.h5"
gain_file = "/sf/alvra/config/jungfrau/gainMaps/JF02T09V02/gains.h5"

with h5py.File(gain_file, "r") as file: 
    G = file["gains"][:]
    
with h5py.File(pede_file, "r") as file: 
    P = file["gains"][:]
    mask = file["pixel_mask"][:]

print("Dimensions of G: ", G.shape)
print("Dimensions of P: ", P.shape)
print("Dimensions of mask: ", mask.shape)

# Detector name for loading module map and applying geometry
detector_name = "JF02T09V02"

Dimensions of G:  (4, 4608, 1024)
Dimensions of P:  (4, 4608, 1024)
Dimensions of mask:  (4608, 1024)


In [4]:
#N_shots_to_use = 5
N_shots_to_use = None

In [5]:
def loadJF_full(DIR, jffilename, G, P, mask, N_shots_to_use, bsreadfile, channel_BS_pulse_ids):
    
    numstds = 2
    minIzero = 0.002
    image_threshold = 1
    hot_pixel = 8
    
    jf_file = h5py.File(DIR + jffilename, "r")

    print(jf_file)

    images = jf_file[f"data/{detector_name}" + channel_JF_images]
    pulse_ids = jf_file[f"data/{detector_name}" + channel_JF_pulse_ids]
    
    try:
        module_maps = jf_file[f"/data/{detector_name}/module_map"]
    except:
        module_maps = None
    
    nframes = images.shape[0]
    
    total = 0
    
    for i in range(nframes):
        image = images[i, :]
        module_map = module_maps[i, :]
        image, mask = apply_module_map(image, module_map, mask)
        image = ju.apply_gain_pede(image, G=G, P=P, pixel_mask=mask, highgain=False)#[roi[0][0]:roi[0][1], roi[1][0]:roi[1][1]]
        image_thr = image.copy()
        image_thr[image < 2] = 0
        total += image_thr
        
    total_corr = ju.apply_geometry(total, detector_name)

    XES = np.sum(total_corr,axis=0)
    
    return XES, total_corr

In [ ]:
for jj in [3,5,8,10,13,14,15,16,17,18]:
    
    filename_root = "RuDimerACN_monoscan_10ps_" + '%03d' % (jj)
    
    print(filename_root)
    
    jffilename = filename_root + '_step00' + str(23) + ".JF02T09V02.h5"
    bsreadfile = filename_root + '_step00' + str(23) + ".BSREAD.h5"

    DIR = "/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_10ps_" + '%03d' % (jj) + "/"
    
    xes, vhimage = loadJF_full(DIR, jffilename, G, P, mask, N_shots_to_use, bsreadfile, channel_BS_pulse_ids)
    print(jj)
    
    if jj == 3:
        VH_sum = vhimage
        XES = xes
    else:
        VH_sum += vhimage
        XES += xes


RuDimerACN_monoscan_10ps_003
<HDF5 file "RuDimerACN_monoscan_10ps_003_step0023.JF02T09V02.h5" (mode r)>
3
RuDimerACN_monoscan_10ps_005
<HDF5 file "RuDimerACN_monoscan_10ps_005_step0023.JF02T09V02.h5" (mode r)>
5
RuDimerACN_monoscan_10ps_008
<HDF5 file "RuDimerACN_monoscan_10ps_008_step0023.JF02T09V02.h5" (mode r)>
8
RuDimerACN_monoscan_10ps_010
<HDF5 file "RuDimerACN_monoscan_10ps_010_step0023.JF02T09V02.h5" (mode r)>
10
RuDimerACN_monoscan_10ps_013
<HDF5 file "RuDimerACN_monoscan_10ps_013_step0023.JF02T09V02.h5" (mode r)>
13
RuDimerACN_monoscan_10ps_014
<HDF5 file "RuDimerACN_monoscan_10ps_014_step0023.JF02T09V02.h5" (mode r)>


In [ ]:
VH_sum = np.load('20_scan_mono_XES.npy', allow_pickle=True)

plt.figure()
plt.imshow(VH_sum, origin="lower", aspect='auto', vmin=0, vmax=10)
plt.colorbar()
plt.title('total_laser_corrected')
plt.tight_layout()
plt.show()